In [1]:
import pandas as pd
import json
import importlib

from tqdm import tqdm
from typing import List, Dict
from pymongo import MongoClient

import boto3
import certifi
import pandas as pd

from utils.extractions_utils import generate_extraction_results, split_text_into_parts, extract_information_from_text, add_extractions_to_splitted_analysis

EndpointConnectionError: Could not connect to the endpoint URL: "https://oidc.us-east-1.amazonaws.com/token"

In [ ]:
STAGE = 'prod'

Loading all key libraries

In [ ]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"{STAGE.capitalize()}/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)

OPENAI_API_KEY = _secrets["openai"]["api_key"]
LLM_API_KEY = _secrets["litellm"]["api_key"]


In [ ]:
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

In [ ]:
BRAND = 'ditp_analysis'

BRAND_DESCR = '''
Feedbacks are from French public services.
'''

## Analysis

In [3]:
def format_ligne(ligne):
    # Fonction interne pour gérer les valeurs manquantes
    def extraire_champ(champ, allow_empty=False):
        return champ if pd.notnull(champ) and (allow_empty or champ != 'N/A') else None

    # Champs obligatoires et facultatifs
    champs = [
        ("Intitulé Structure 1", ligne.get("intitule_structure_1"), False),
        ("Intitulé Structure 2", ligne.get("intitule_structure_2"), True),
        ("Tags Métiers", ligne.get("tags_metiers"), True),
        ("Pays de la demande", ligne.get("pays"), False),
        ("Feedback complet", ligne.get("verbatims"), False),
    ]
    
    # Initialisation des lignes avec une phrase fixe
    lignes = ["Feedbacks are from French public services."]
    
    # Génération des lignes dynamiques si les champs sont présents
    for label, champ, allow_empty in champs:
        valeur = extraire_champ(champ, allow_empty)
        if valeur:
            lignes.append(f"{label}: {valeur}")
    
    # Retour du résultat formaté
    return "\n".join(lignes)

In [4]:
# Keeping only feedbacks with extractions and splitted_analysis_v2

# subdf = from_mongo[from_mongo['extractions'].notna()]
# subdf = subdf[subdf['splitted_analysis_v2'].notna()]
df = pd.read_csv('ditp_test_2.csv')
subdf = df
subdf['brand_context'] = subdf.apply(format_ligne, axis=1)
print(subdf.shape)

row = subdf.sample().iloc[0]

print("Example of row:", row.verbatims)


(43, 85)
Example of row: Site internet de la Caisse des allocations familiales
Allocataire de la Caisse des Allocations Familiales, je me suis connectée à mon espace en ligne. Deux formulaires de déclarations de revenue m'ont été adressés à compléter.Après avoir rempli les informations demandées par les formulaires, j'ai transféré le fichier en cliquant sur l'encart destiné au transfère du fichier à mon espace allocataire en ligne. Un dysfonctionnement du site internet www.caf.fr a enregistré le premier formulaire pour toute les demandes de déclarations année 2018, année 2017 et année 2016.Le site internet de la Caisse des Allocations Familiales inscrit pourtant que je n'ai pas répondu à ces demandes de déclaration de revenue. Une inscription alerte reste affichéCes déclarations de revenues avaient déjà été remis en format papier à la Caisse des Allocations Familiales. La déclaration avait aussi été effectué en ligne.


The function `classify_feedback_extractions` is used to classify each extraction.

In [5]:
split_text_into_parts(row.verbatims)

['Site internet de la Caisse des allocations familiales',
 'Allocataire de la Caisse des Allocations Familiales',
 'je me suis connectée à mon espace en ligne',
 "Deux formulaires de déclarations de revenue m'ont été adressés à compléter",
 'Après avoir rempli les informations demandées par les formulaires',
 "j'ai transféré le fichier en cliquant sur l'encart destiné au transfère du fichier à mon espace allocataire en ligne",
 'Un dysfonctionnement du site internet www',
 'caf',
 'fr a enregistré le premier formulaire pour toute les demandes de déclarations année 2018',
 'année 2017 et année 2016',
 "Le site internet de la Caisse des Allocations Familiales inscrit pourtant que je n'ai pas répondu à ces demandes de déclaration de revenue",
 'Une inscription alerte reste affichéCes déclarations de revenues avaient déjà été remis en format papier à la Caisse des Allocations Familiales',
 'La déclaration avait aussi été effectué en ligne']

In [ ]:
print(row["_id"])
print(row["extractions"])
print(row["brand_context"])

print(type(row["_id"]))
print(type(row["extractions"]))
print(type(row["brand_context"]))

ditp_analysis/3709291
[{'sentiment': 'NEGATIVE', 'extraction': "Refus de prime vélo en raison d'une facture manquante", 'text': 'prime velo\n'}, {'sentiment': 'NEGATIVE', 'extraction': "Refus de prime vélo en raison d'une facture manquante", 'text': ' après un refus ,'}, {'sentiment': 'NEGATIVE', 'extraction': "Refus de prime vélo en raison d'une facture manquante", 'text': 'car il manquait une facture ,'}, {'sentiment': 'NEGATIVE', 'extraction': "Absence de nouvelles après avoir corrigé l'erreur", 'text': " j'ai remédier a cette erreur un mois 1 mois et demie après toujours pas de nouvelle et bien sur pas de virement"}, {'sentiment': 'NEGATIVE', 'extraction': 'Pas de virement reçu', 'text': " j'ai remédier a cette erreur un mois 1 mois et demie après toujours pas de nouvelle et bien sur pas de virement"}]
Feedbacks are from French public services.
Intitulé Structure 1: Aides aux véhicules
Intitulé Structure 2: Agence de services et de paiement (ASP)
Tags Métiers: Dépôt de dossier
Pays

In [6]:
from utils.database import (
    get_elementary_subjects,
    remove_elementary_subject_from_mongo,
    push_new_elementary_subject_to_mongo,
    get_one_elementary_subject,
    get_one_topic,
    get_one_classification_scheme,
    get_all_classification_schemes,
    update_feedback_in_mongo,
    get_feedbacks_with_extractions,
    get_field_value
)
get_all_classification_schemes('columbuscafe')


[]

In [7]:
from utils.topics_utils import (get_most_occuring_elementary_subjects)

get_most_occuring_elementary_subjects('columbuscafe')

[{'_id': 'Personnel agréable', 'count': 878},
 {'_id': 'Ambiance agréable et détendue', 'count': 749},
 {'_id': 'Cadre agréable', 'count': 577},
 {'_id': 'Accueil chaleureux', 'count': 511},
 {'_id': 'Personnel souriant', 'count': 455},
 {'_id': 'Personnel aimable', 'count': 412},
 {'_id': 'Accueil souriant', 'count': 370},
 {'_id': 'Manque de professionnalisme du personnel', 'count': 321},
 {'_id': 'Qualité du café', 'count': 294},
 {'_id': 'Qualité de la nourriture', 'count': 256},
 {'_id': 'Qualité des muffins', 'count': 244},
 {'_id': 'Qualité des pâtisseries', 'count': 242},
 {'_id': 'Personnel pas souriant, aimable ou accueillant', 'count': 237},
 {'_id': 'Rapidité du service', 'count': 235},
 {'_id': 'Propreté', 'count': 217},
 {'_id': 'Qualité des boissons', 'count': 210},
 {'_id': 'Attente prolongée pour passer commande', 'count': 198},
 {'_id': 'Personnel accueillant et sympathique', 'count': 194},
 {'_id': 'Prix abordables', 'count': 189},
 {'_id': 'Qualité générale de la no

In [26]:
from utils.database import (
    get_elementary_subjects,
    remove_elementary_subject_from_mongo,
    push_new_elementary_subject_to_mongo,
    get_one_elementary_subject,
    get_one_topic,
    get_one_classification_scheme,
    get_all_classification_schemes,
    update_feedback_in_mongo,
    get_feedbacks_with_extractions,
    get_field_value
)
p = get_elementary_subjects('ditp_analysis', 'negative')


In [27]:
h= p[:50]

In [28]:
for i in h:
    print(i['elementary_subject'])

Suivi de dossier : Accès aux documents de rejet
Accessibilité : Absence de porte-document
Suivi de dossier : Absence de justification de rejet


In [69]:
from utils.database import remove_all_elementary_subjects_from_mongo
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'negative')
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'positive')
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'suggestion')
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'NEGATIVE')
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'POSITIVE')
remove_all_elementary_subjects_from_mongo('ditp_analysis', 'SUGGESTION')

In [15]:
p = [{'_id': '670daa7e19a9e6360a7f6246', 'brand': 'ditp_analysis', 'id': '_elementary_subjects_', 'type': 'SUGGESTION', 'elementary_subject': 'Paiement : Retard de paiement pour stages', 'embeddings': [[-0.01244667824357748, -0.02859530970454216, -0.006363960914313793, -0.026894764974713326, -0.014820899814367294, 0.005664121359586716, -0.01436306070536375, 0.011844947002828121, -0.011897271499037743, -0.018666746094822884, 0.026633143424987793, 0.023218972608447075, -0.0033618456218391657, -0.005441742483526468, -0.015943361446261406, 0.003142869332805276, -0.002027028938755393, -0.020660102367401123, -0.022817067801952362, -0.024363327771425247]], 'mappings': []}]


In [18]:
subject_names = [subject["elementary_subject"] for subject in p]
subject_embeddings = [subject["embeddings"] for subject in p]


In [19]:
subject_names

['Paiement : Retard de paiement pour stages']

In [36]:
from utils.database import (get_elementary_subjects)
existing_subjects = get_elementary_subjects('ditp_analysis', 'negative')